In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
folder_list = []
train_dir = Path('train')

for folder in train_dir.iterdir():
    folder_list.append(folder)
folder_list


[WindowsPath('train/00001'),
 WindowsPath('train/00002'),
 WindowsPath('train/00003'),
 WindowsPath('train/00004'),
 WindowsPath('train/00005'),
 WindowsPath('train/00006'),
 WindowsPath('train/00007'),
 WindowsPath('train/00008'),
 WindowsPath('train/00009'),
 WindowsPath('train/00010')]

In [5]:
training_dir = []

for folder in folder_list:
    if folder.is_dir():
        training_dir.append(folder/'train_stage1.csv')
training_dir

[WindowsPath('train/00001/train_stage1.csv'),
 WindowsPath('train/00002/train_stage1.csv'),
 WindowsPath('train/00003/train_stage1.csv'),
 WindowsPath('train/00004/train_stage1.csv'),
 WindowsPath('train/00005/train_stage1.csv'),
 WindowsPath('train/00006/train_stage1.csv'),
 WindowsPath('train/00007/train_stage1.csv'),
 WindowsPath('train/00008/train_stage1.csv'),
 WindowsPath('train/00009/train_stage1.csv'),
 WindowsPath('train/00010/train_stage1.csv')]

Random Forest

In [28]:
from sklearn.model_selection import KFold
from sklearn.metrics import brier_score_loss

def set_label(row):
    probabilities = row[2:]
    max_prob_index = probabilities.idxmax()
    return max_prob_index

train_dataframes = [pd.read_csv(f) for f in training_dir]

kf = KFold(n_splits=5)

scores = []

counts = 0

for train_indices, val_indices in kf.split(train_dataframes):
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

    X_train = pd.concat([train_dataframes[i].drop(['location', 'label'], axis=1) for i in train_indices], ignore_index=True)
    X_train['room'] = X_train['room'].replace({'living_room':1, 'hallway':2, 'kitchen':3})
    y_train = pd.concat([train_dataframes[i]['label'] for i in train_indices], ignore_index=True)
    
    X_val = pd.concat([train_dataframes[i].drop(['location', 'label'], axis=1) for i in val_indices], ignore_index=True)
    X_val['room'] = X_val['room'].replace({'living_room':1, 'hallway':2, 'kitchen':3})
    y_val = pd.concat([train_dataframes[i]['label'] for i in val_indices], ignore_index=True)

    rf_model.fit(X_train, y_train)

    # for train_idx in traini_indices:
    #     train_data = train_dataframes[train_idx]
    #     X_train = train_data.drop(['location', 'label'], axis=1)
    #     X_train['room'] = X_train['room'].replace({'living_room':1, 'hallway':2, 'kitchen':3})
    #     y_train = train_data['label']
    #     y_train = y_train.loc[y_train != 0]
    #     rf_model.n_estimators += 100
    #     rf_model.fit(X_train, y_train)

    
    # for val_idx in val_indices:
    #     val_data = train_dataframes[val_idx]
    #     X_val = val_data.drop(['location', 'label'], axis=1)
    #     X_val['room'] = X_val['room'].replace({'living_room':1, 'hallway':2, 'kitchen':3})
    #     y_val = val_data['label']
    #     y_val = y_val.loc[y_val != 0]

    #     y_pred = rf_model.predict(X_val)
    
    # val_df = pd.concat([train_dataframes[i] for i in val_indices], ignore_index=True)
    # X_val = val_df.drop(['location', 'label'], axis=1)
    # X_val['room'] = X_val['room'].replace({'living_room':1, 'hallway':2, 'kitchen':3})
    # y_val = val_df['label']

    y_preds = rf_model.predict(X_val)

    acc = accuracy_score(y_val, y_preds)
    scores.append(acc)

    counts += 1
    print(f'Fit {counts}: Accuracy = {acc}')


print(f'Accuracy mean: {np.mean(scores)}')



Fit 1: Accuracy = 0.5455349248452697
Fit 2: Accuracy = 0.5240746806419915
Fit 3: Accuracy = 0.5962732919254659
Fit 4: Accuracy = 0.5592686002522068
Fit 5: Accuracy = 0.5103964513446078
Accuracy mean: 0.5471095898019083
